In [117]:
# Advent of Code 2023
# Day 10 Problem 1
import re
from collections import Counter
import functools

with open("aoc_10_input.txt") as f:
    A = f.read().strip().split("\n")

# c+p test case here
TEST = """..F7.
.FJ|.
SJ.L7
|F--J
LJ...""".strip().split("\n")

TESTB = """7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ""".strip().split("\n")

for i, line in enumerate(TEST):
    print(f"{i}: {line}")


0: ..F7.
1: .FJ|.
2: SJ.L7
3: |F--J
4: LJ...


In [122]:
# my first attempt is recursive and hits the recursion limit
# so I will reimplement as an iterative solution.
# however, these three are used between both attempts
def parseInput(lines):
    L = len(lines[0])
    res = [["." for _ in range(L+2)]]
    for line in lines:
        temp = list(line)
        temp.append(".")
        temp.insert(0, ".")
        res.append(temp)
    res.append(["." for _ in range(L+2)])
    return res

def findS(inp):
    for i, row in enumerate(inp):
        for j, char in enumerate(row):
            if char == "S":
                return (i,j)
    return (-1,-1)

def findNeighbors(M, x, y):
    res = []
    # north
    if M[x-1][y] in "|7F":
        res.append([x-1, y])
    # south
    if M[x+1][y] in "|LJ":
        res.append([x+1, y])
    # west
    if M[x][y-1] in "-LF":
        res.append([x, y-1])
    # east
    if M[x][y+1] in "-7J":
        res.append([x, y+1])
    return res


In [123]:
def goDown(M,x,y,count):
    if M[x][y] == "S":
        return count
    if M[x][y] == "|":
        return goDown(M,x+1,y,count+1)
    if M[x][y] == "L":
        return goRight(M,x,y+1,count+1)
    if M[x][y] == "J":
        return goLeft(M,x,y-1,count+1)
    return 0

def goUp(M,x,y,count):
    if M[x][y] == "S":
        return count
    if M[x][y] == "|":
        return goUp(M,x-1,y,count+1)
    if M[x][y] == "F":
        return goRight(M,x,y+1,count+1)
    if M[x][y] == "7":
        return goLeft(M,x,y-1,count+1)
    return 0

def goRight(M,x,y,count):
    if M[x][y] == "S":
        return count
    if M[x][y] == "-":
        return goRight(M,x,y+1,count+1)
    if M[x][y] == "7":
        return goDown(M,x+1,y,count+1)
    if M[x][y] == "J":
        return goUp(M,x-1,y,count+1)
    return 0

def goLeft(M,x,y,count):
    if M[x][y] == "S":
        return count
    if M[x][y] == "-":
        return goLeft(M,x,y-1,count+1)
    if M[x][y] == "F":
        return goDown(M,x+1,y,count+1)
    if M[x][y] == "L":
        return goUp(M,x-1,y,count+1)
    return 0

def initialDirection(M, sx, sy, nx, ny):
    # same row
    if sx == nx:
        # west
        if ny == sy - 1:
            print("Go left!")
            return goLeft(M,nx,ny,1)
        # east
        elif ny == sy + 1:
            print("Go right!")
            return goRight(M,nx,ny,1)
    # same col
    else:
        # north
        if nx == sx - 1:
            print("Go up!")
            return goUp(M,nx,ny,1)
        elif nx == sx + 1:
            print("Go down!")
            return goDown(M,nx,ny,1)
    return 0

# this hits the recursion limit so I will try out an iterative version instead
def p1(inp):
    tapestry = parseInput(inp)
    #for row in tapestry:
    #    print(row)
    START = findS(tapestry)
    print(f"Starting point is at {START}\n")
    startNeighbors = findNeighbors(tapestry, *START)
    print(f"Starting neighbors: {startNeighbors}")
    res = initialDirection(tapestry, *START, *startNeighbors[0])
    print(f"{res}//2 = {res//2}")
    return res//2


In [119]:
def traverse(M, stack):
    while(stack[-1] != stack[0]):
        px, py = stack[-1][0], stack[-1][1]
        diff = [px - stack[-2][0], py - stack[-2][1]]
        # we are going south
        if diff == [1, 0]:
            if M[px][py] == "|":        nextPos = [px+1, py]    # continue going south
            elif M[px][py] == "L":      nextPos = [px, py+1]    # go east
            elif M[px][py] == "J":      nextPos = [px, py-1]    # go west           
        # we are going north
        elif diff == [-1,0]:
            if M[px][py] == "|":        nextPos = [px-1, py]    # continue going north
            elif M[px][py] == "F":      nextPos = [px, py+1]    # go east
            elif M[px][py] == "7":      nextPos = [px, py-1]    # go west
        # we are going east
        elif diff == [0,1]:
            if M[px][py] == "-":        nextPos = [px, py+1]    # continue going east
            elif M[px][py] == "J":      nextPos = [px-1, py]    # go north
            elif M[px][py] == "7":      nextPos = [px+1, py]    # go south
        # we are going west
        elif diff == [0,-1]:
            if M[px][py] == "-":        nextPos = [px, py-1]    # continue going west
            elif M[px][py] == "L":      nextPos = [px-1, py]    # go north
            elif M[px][py] == "F":      nextPos = [px+1, py]    # go south            
        stack.append(nextPos)
    return stack[:-1]

def p1_take2(inp):
    tapestry = parseInput(inp)
    startPos = findS(tapestry)
    startNeighbors = findNeighbors(tapestry, *startPos)
    stack = [list(startPos), startNeighbors[0]]
    completePath = traverse(tapestry, stack)
    return len(completePath)//2


In [120]:
# expected output: 8
#print(p1(TEST))
print(p1_take2(TESTB))



8


In [121]:
p1_take2(A)


6890

In [109]:
# Part 2
def transformTapestry(M, S):
    res = []
    for i,row in enumerate(M):
        r = []
        for j,node in enumerate(row):
            if [i,j] in S:
                r.append(" ")
            else:
                r.append("o")
        res.append(r)
    
    #for row in res:
    #    print("".join(row))
    #print()
    return floodOutside(res)

def floodOutside(tap):
    queue = [[0,0]]
    H, W = len(tap)-1, len(tap[0])-1
    while queue:
        nx,ny = queue.pop()
        tap[nx][ny] = " "
        if nx > 0 and tap[nx-1][ny] == "o":
            queue.append([nx-1,ny])
        if ny > 0 and tap[nx][ny-1] == "o":
            queue.append([nx,ny-1])
        if nx < H and tap[nx+1][ny] == "o":
            queue.append([nx+1,ny])
        if ny < W and tap[nx][ny+1] == "o":
            queue.append([nx,ny+1])
    res = 0
    for row in tap:
        R = "".join(row)
        #print(R)
        res += R.count("o")
    return res

def p2(inp):
    tapestry = parseInput(inp)
    print(len(tapestry), len(tapestry[0]))
    startPos = findS(tapestry)
    #print(f"Starting point is at {startPos}\n")
    startNeighbors = findNeighbors(tapestry, *startPos)
    stack = [list(startPos), startNeighbors[0]]
    #print(stack)
    completePath = traverse(tapestry, stack)
    return transformTapestry(tapestry, completePath)

In [110]:
TESTC = """FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L""".strip().split("\n")


In [111]:
p2(TESTC)

12 22


10

In [112]:
# 720 is too high
# 575 too high
# 280 too low
p2(A)

142 142


720

In [113]:
tap = """oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo      ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooooooooooooooooooooooooooooooo  oooo      ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooooooooooooooooooooooooooooooo   ooo      ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooooooooooooooooooo  oooooooooo    ooo     oo  ooooo  oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooooooooooooooooooo  ooooooooooo   ooo   oooo  ooooo  oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooooo    oooooo  o    ooo   o     ooo     ooooooooooooooooooo  oooooooooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooooo    ooo          ooo         ooo       ooooooooooooooooo  o     oooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooo  ooooooo    oo        o  o             o      ooooooooooooooooo        oooooooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooooo  oo   ooooooo     oo             o          o      oooooooooo   oo          oooooooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooooo  oo   oooo  ooo                            oo      oooooooooo       o       oooooooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooo         ooo                                  ooo      oooooooooo         o    oo  oooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooo                                              o        ooo  oooooo        o        oooooooooooooooooooooooooooooooo
oooooooooooooooooooooo   o  o  o       o                               o           oo  oooooo                    ooooooooooooo  oooooooooooooo
oooooooooooooooooooooo                                                                  ooo                      oooo  oooooo    ooooooooooooo
oooooooooooooooo  ooo                                o    o                     o       ooo                o            oooo     ooooooooooooo
oooooooooooooooo  o                o                               o                       o                            oo     ooooooooooooooo
ooooooooooooo     o                                                                        o                            oo       ooo  oooooooo
ooooooooooooo                                                                                              o            o        oo    ooooooo
oooooooooooooo                        o                                        o                                       o          o    ooooooo
ooooooooooooooo                                 o  o  oo      o     o                                  o                              oooooooo
oooooooooooooooo            o                       o  o       o              o                                                       oooooooo
oooooooooooooooo                                                   o                                                                   ooooooo
ooooooooooooooooo              o                 o                 o                          o            o                           ooooooo
oooooooooooooooo                  o                            o           o                                                   o     ooooooooo
oooooooooooooooo                                                                                                                     ooooooooo
oooooooooooooooo      o                                          o        o     o                                                      ooooooo
ooooooooooooooo                               o                            o                                                   o       ooooooo
ooooooooooooooo                                    o  o                                   o                                         oooooooooo
ooooooooooooo         o                                      o                                     o         o            o          ooooooooo
ooooooooooooo                                            o          o      o                               o                   o       ooooooo
oooooooooooo                                                                        o                     o             o    o    o    ooooooo
oooooooooooo                                                                                                                          oooooooo
oooooooooooooo                                     o                        o                                o                        oooooooo
oooooooooooo    o  o                o                                                                                                 oooooooo
ooooooooooo     o                       o               o       o                                                   o            o  oooooooooo
ooooooooooo                                                                                                    o                    oooooooooo
oooooooooooooo       o                               o                                                                            oooooooooooo
oooooooooooooo                                                                                                        o         oooooooooooooo
oooooooooooooo                                    o                   o                                  o                      oooooooooooooo
oooooooooooooo                                                                                   o                         o    ooo  ooooooooo
oooooooooooooo                       o  o                                                                                        o   ooooooooo
oooooooooooooooo    o          o                          o                                                                          ooooooooo
oooooooooooooooo                         o                                                o                            oo       o   oooooooooo
oooooooooooooo                o              o                                           o               o                    o    ooooooooooo
oooooooooooooo      o                                                      o  o                                              oo      ooooooooo
ooooooooooooooo                    o               o           o                      o     o                                 o      ooooooooo
ooooooooooooooo                      o       o               o                                                                         ooooooo
oooooooooooooo  o                                                          o                          o                                ooooooo
oooooooooooooo              o               o     o                                                    o                             ooooooooo
ooooooooooooo                   o                        o                o                 o  o                 o      o    o       ooooooooo
ooooooooooo     o  o           o             o                   o                   o                                                 ooooooo
ooooooooooo    o                                                                                                                       ooooooo
ooooooooooo                                            o                                  o                               o            ooooooo
ooooooooooo                      o                                                                                                     ooooooo
oooooooooooooo      o                                                 o                                                o     o        oooooooo
ooooooooooooo                 o                 oo          o             o               o                                            ooooooo
ooooooooooooo                                             o                                                                            ooooooo
oooooooooo     o                             o      o              o           o                              o                     o  ooooooo
oooooooooo                                                 oo      oooo  o     o                                  o            o  oooooooooooo
oooooooo                                             o  oo         ooooooo     o                                     o            oooooooooooo
oooooooo                                                           oooooooooo                                                     oooooooooooo
oooooooo                       o                                 oooooooooooo                        o          o               oooooooooooooo
oooooooo                                                     oooooooooooooooo                                                   oooooooooooooo
ooooo  o                                                      ooooooooooooooooo                                                ooo   ooooooooo
oooo                             o                             oooooooooooooooo        o       o                  o             oo   ooooooooo
oooo         o  o                        o                     oooooooooooooooo                                o                oo    oooooooo
oooooo                                    o                     ooooooooooooooooo           o   o  o                           ooo    oooooooo
oooooo                                                          oooooooooooooooooooo                                           ooo  oooo  oooo
oooooo                                                      o  oooooooooooooooooooo                                  o                 o  oooo
ooooooooo                                                       ooooooooooooooooooo                                                    o  oooo
oooooooooo                                                      oooooooooooooooooo                         o          o               oo  oooo
oooooooooo  o                                           o     oooooooooooooooooooo                                                    o      o
ooooo    o                                                      oooooooooooooooo                                                             o
ooooo                                                           oooooooooooooooo                                          o              oo  o
ooooooo   o              o           o                           o    ooooooooooooo               o           o                          ooooo
oooooooo          o  o                  o     o                       ooooooooooooo                                                      ooooo
ooooo               o           o                                     ooooooooo          o         o                                       ooo
ooooo                                o                               oooooooooo                                                            ooo
oooooooo                                            o                 oooo                                         o                    oooooo
ooooooooo          o        o                                o  o     oooo      oo                     o                                oooooo
oooooooooooo  o                                                      ooooooo    o                                      o              oooooooo
ooooooooooo                                                          oo    o                                       o                  oooooooo
ooooooooooo           o                 o                         o  oo                                         o       o      o    oooooooooo
oooooooooooo                                           o          ooo           o                                 o                  ooooooooo
oooooooooooo                         o             o  oo           o       o  o                                                      ooooooooo
oooooooooooooo                                         o  o                         o                                  o            oooooooooo
oooooooooooooo   o                       o                                              o          o     o                          oooooooooo
oooooooooooooo            o                                        o                                       o                       ooooooooooo
oooooooooooooo                                    o       o  o      o   o                                                        ooooooooooooo
ooooooooooooooo                          o                                   o       o                   o                          oooooooooo
ooooooooooooooo   o                         o    o                   o                                       o         o             ooooooooo
ooooooooooooo            o    o  o            o                     o                             o  o                    o  o       ooooooooo
oooooooooooo                                                                                                                         o  oooooo
oooooooooooo                                                                                                           o                 ooooo
oooooooooooo                               o             o                                o                                 o            ooooo
oooooooooooo                o                           o                                     o      o                                  oooooo
ooooooooooo                                 o                              o                                                      o     oooooo
ooooooooo                                          o                                        o              o                           ooooooo
ooooooooo        o                                                                       o           o                   o     o     ooooooooo
oooooooooooo           o                                 o                              o                                            ooooooooo
oooooooooooooooo                       o        o                       o                                  o                              oooo
oooooooooooooooo                                                                     o               o                                    oooo
ooooooooooooo                  o                                                            o                                     o      ooooo
ooooooooooooo                  o     o                              o                                      o          o                  ooooo
oooooooooooooooo                       o         o          o                                      o                                     ooooo
ooooooooooooooooo                                                                                                                       oooooo
oooooooooooooooo                         o                            o                           o                                   oooooooo
ooooooooooooooo                                                                                                                       oooooooo
ooooooooooooooo                                            o             o                                      o                oo   oooooooo
oooooooooooooooooo                                           o                  o             o                                  oo   oooooooo
oooooooooooooooo  o                                                                                        o           o                 ooooo
oooooooooooooooo      o                   o                 o                                                         o                  ooooo
oooooooooooooooo              o                    o               o                                                            oooo  oooooooo
ooooooooooooooooo                      o                                             o                             o            oooo  oooooooo
ooooooooooooooooo         o      o            o           o         o         o             o                       o         oooooooooooooooo
ooooooooooooooooooo                      o                                                                    o              ooooooooooooooooo
ooooooooooooooooooo                                   o                                                                     oooooooooooooooooo
ooooooooooooooooooo      o                                                              o              o                    oooooooooooooooooo
ooooooooooooooooooo    o     o      o    o                           o                                       o       o      oooooooooooooooooo
ooooooooooooooooooo                               o                               o                                 o       oooooooooooooooooo
ooooooooooooooooooooooo                                                            oo   o    o                                oooooooooooooooo
ooooooooooooooooooooooo                      o                 o               o    o                            o     ooo    oooooooooooooooo
ooooooooooooooooooooooo                                                                                   o          ooooooooooooooooooooooooo
ooooooooooooooooooooooooo                       o                                                             o      ooooooooooooooooooooooooo
ooooooooooooooooooooooooo       o                                o                     o                          oooooooooooooooooooooooooooo
oooooooooooooooooooooooooo                                                       o                                 ooooooooooooooooooooooooooo
oooooooooooooooooooooooooooo     o  oo        o                                                                o    oooooooooooooooooooooooooo
ooooooooooooooooooooooooooooooooooo     oo                                                                    ooo    ooooooooooooooooooooooooo
ooooooooooooooooooooooooooooooooooo     ooo                                        o  o         o      o  o   oooo   ooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooo      o   oo    oo  o                                oo  ooooo  o  oooooooooooo  ooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooo       oo      o                     o             oooooooooooooooooooooooooo  ooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooo     oo                         o               oooooooooooooooooooooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooooooooooo                                        oooooooooooooooooooooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooooooooooooo                 o       o        oooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
ooooooooooooooooooooooooooooooooooooooooooooooooooo     o                 ooo        ooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooooooooooooooo  o                 ooooo        ooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooooooooooooooo  o              oooooooooo   oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooooooooooooooooooo              oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooooooooooooooooooo               ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooooooooooooooooooooo  ooo  o     ooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
oooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo
""".strip().split("\n")

In [114]:
R = [row[36:106] for row in tap[36:106]]

In [116]:
res = 0
for row in R:
    res += "".join(row).count("o")
print(res)

453
